In [7]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))

torch.rand(size=(2,4))
net(X)


tensor([[-0.2209],
        [-0.1918]], grad_fn=<AddmmBackward0>)

In [8]:
#我们要把每一层的参数，也就是Sequential拿出来，把权重也就是状态拿出来，全联接层有两个参数，weight和bias
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0685,  0.2738,  0.1639, -0.2915, -0.0094,  0.1908, -0.3431,  0.1480]])), ('bias', tensor([-0.1107]))])


In [10]:
#可以访问一个具体的参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1107], requires_grad=True)
tensor([-0.1107])


In [11]:
#访问这个weight的梯度，这个地方还没有反向计算
print(net[2].weight.grad)

None


In [12]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
#这个可以拿出字符串的名字和对应的形状
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [13]:
net.state_dict()['2.bias'].data

tensor([-0.1107])

In [14]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    #初始化这个net
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套，每个net加入一个块，第一个参数是块的名字，第二个参数是块
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.4379],
        [0.4380]], grad_fn=<AddmmBackward0>)

In [16]:
#可以通过print来看这个网络的结构，这是三个Sequential进行嵌套
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


<h3>内置初始化</h3>

In [17]:
def init_normal(m):
    #如果为线性类，也就是全联接层的话
    if type(m) == nn.Linear:
        #下划线在后面说明这是一个替换函数，不是一个值，而是会把这些替换掉
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

#net中所有的这些都调用这个，如果是嵌套的话也会嵌套
net.apply(init_normal)
#这里所有的权重和偏移都会变化
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0217, -0.0067, -0.0033, -0.0096]), tensor(0.))

In [18]:
#初始化一个唯一的值
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [19]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
#可以对每一层进行独自的初始化
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5269, -0.3513,  0.2660,  0.6020])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


<h3>参数绑定</h3>
这个大部分是有些层需要共享权重会用到，可以减少内存消耗

In [20]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
